In [1]:
import os
import shutil
from torchvision import datasets, transforms, models
from torch.utils.data import DataLoader
import torch
import torch.nn as nn
import torch.optim as optim

# 資料增強與標準化
transform = transforms.Compose([
    transforms.Resize(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

In [2]:
# 加載數據
train_dataset = datasets.ImageFolder(root='../dataloader_c23/train', transform=transform)
val_dataset = datasets.ImageFolder(root='../dataloader_c23/validation', transform=transform)
test_dataset = datasets.ImageFolder(root='../dataloader_c23/test', transform=transform)

train_loader = DataLoader(train_dataset, batch_size=256, shuffle=True, num_workers=2)
val_loader = DataLoader(val_dataset, batch_size=256, shuffle=False, num_workers=2)
test_loader = DataLoader(test_dataset, batch_size=256, shuffle=False, num_workers=2)

In [39]:
import timm

# 設定設備
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print("using {} device.".format(device))

# 創建模型，預訓練且將分類層設置為2類
model = timm.create_model('efficientformerv2_s0', pretrained=True)
model.head = nn.Linear(model.head.in_features, 2)
model.head_dist = nn.Linear(model.head_dist.in_features, 2)
model = model.to(device)

# 定義損失函數和優化器
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

using cuda device.


In [26]:
# 訓練函數
def train(epoch, epochs, model, train_loader, optimizer, loss_function, device):
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0
    train_bar = tqdm(train_loader, file=sys.stdout)
    
    for step, data in enumerate(train_bar):
        images, labels = data
        images, labels = images.to(device), labels.to(device)  # 確保數據在正確的設備上
        optimizer.zero_grad()
        logits = model(images)  # 模型輸出
        loss = loss_function(logits, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

        # 計算準確率
        _, predicted = torch.max(logits, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

        train_bar.desc = "train epoch[{}/{}] loss:{:.3f}".format(epoch + 1, epochs, loss)
    running_loss = running_loss / train_steps
    accuracy = 100. * correct / total
    return running_loss, accuracy

In [27]:
# 驗證函數
def validate(epoch, epochs, model, validate_loader, loss_function, device):
    model.eval()
    acc = 0.0
    val_loss = 0.0
    val_num = len(validate_loader.dataset)
    
    with torch.no_grad():
        val_bar = tqdm(validate_loader, file=sys.stdout)
        for val_data in val_bar:
            val_images, val_labels = val_data
            val_images, val_labels = val_images.to(device), val_labels.to(device)
            outputs = model(val_images)
            loss = loss_function(outputs, val_labels)
            val_loss += loss.item() * val_images.size(0)

            predict_y = torch.max(outputs, dim=1)[1]
            acc += torch.eq(predict_y, val_labels).sum().item()

            val_bar.desc = "valid epoch[{}/{}]".format(epoch + 1, epochs)
    
    val_loss /= val_num
    val_accurate = acc / val_num
    return val_loss, val_accurate

In [28]:
import time
import sys
from tqdm import tqdm
# 訓練和驗證模型
num_epochs = 25
best_acc = 0.0
train_steps = len(train_loader)
save_path = 'best_mobilenetV4_s.pth'
t_l, t_a = [], []
v_l, v_a = [], []
for epoch in range(num_epochs):
    start_time = time.time()
    train_loss, train_accuracy = train(epoch, num_epochs, model, train_loader, optimizer, criterion, device)
    val_loss, val_accurate = validate(epoch, num_epochs, model, val_loader, criterion, device)
    t_l.append(train_loss)
    t_a.append(train_accuracy)
    v_l.append(val_loss)
    v_a.append(val_accurate)
    
    print('[epoch %d] train_loss: %.3f  train_accuracy: %.3f' %
            (epoch + 1, train_loss, train_accuracy))
    print('[epoch %d] val_loss: %.3f  val_accuracy: %.3f' %
            (epoch + 1, val_loss, val_accurate))
    
    if val_accurate > best_acc:
        best_acc = val_accurate
        torch.save(model.state_dict(), save_path)
    end_time = time.time()
    print(f'Training_Time: {end_time - start_time:.2f} seconds')
    
print('訓練完成')

valid epoch[1/25]: 100%|██████████| 55/55 [00:27<00:00,  2.02it/s]
[epoch 1] train_loss: 0.461  train_accuracy: 76.817
[epoch 1] val_loss: 0.477  val_accuracy: 0.808
Training_Time: 193.88 seconds
valid epoch[2/25]: 100%|██████████| 55/55 [00:22<00:00,  2.41it/s]
[epoch 2] train_loss: 0.184  train_accuracy: 92.636
[epoch 2] val_loss: 0.810  val_accuracy: 0.733
Training_Time: 157.82 seconds
valid epoch[3/25]: 100%|██████████| 55/55 [00:22<00:00,  2.49it/s]
[epoch 3] train_loss: 0.110  train_accuracy: 95.688
[epoch 3] val_loss: 0.409  val_accuracy: 0.859
Training_Time: 155.73 seconds
valid epoch[4/25]: 100%|██████████| 55/55 [00:22<00:00,  2.41it/s]
[epoch 4] train_loss: 0.077  train_accuracy: 97.104
[epoch 4] val_loss: 0.454  val_accuracy: 0.859
Training_Time: 156.87 seconds
valid epoch[5/25]: 100%|██████████| 55/55 [00:22<00:00,  2.41it/s]
[epoch 5] train_loss: 0.060  train_accuracy: 97.743
[epoch 5] val_loss: 0.769  val_accuracy: 0.825
Training_Time: 155.96 seconds
valid epoch[6/25]: 1

In [31]:
# 測試模型
model.load_state_dict(torch.load('best_efficientformerV2_s0.pth'))
model.eval()
test_running_corrects = 0

with torch.no_grad():
    for inputs, labels in test_loader:
        inputs = inputs.to(device)
        labels = labels.to(device)

        outputs = model(inputs)
        _, preds = torch.max(outputs, 1)
        test_running_corrects += torch.sum(preds == labels.data)

test_acc = test_running_corrects.double() / len(test_dataset)
print(f'Test Acc: {test_acc:.4f}')

Test Acc: 0.8776


In [32]:
def save_metrics_to_file(t_l, t_a, v_l, v_a, filename='metrics.txt'):
    with open(filename, 'w') as file:
        file.write("Train Loss:\n")
        for item in t_l:
            file.write(f"{item}\n")
        
        file.write("Train Accuracy:\n")
        for item in t_a:
            file.write(f"{item}\n")
        
        file.write("Validation Loss:\n")
        for item in v_l:
            file.write(f"{item}\n")
        
        file.write("Validation Accuracy:\n")
        for item in v_a:
            file.write(f"{item}\n")

In [33]:
# 假設 t_l, t_a, v_l, v_a 已經被填充
filename = 'efficientformerV2_s0.txt'
save_metrics_to_file(t_l, t_a, v_l, v_a, filename)

In [ ]:
import torch
import torch.nn as nn
import timm
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

# 確認設備
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# 創建模型，預訓練且將分類層設置為2類
model = timm.create_model('efficientformerv2_s0', pretrained=True)
model.head = nn.Linear(model.head.in_features, 2)
model.head_dist = nn.Linear(model.head_dist.in_features, 2)
model = model.to(device)

# 打印模型結構以確認最後一層
print(model)

In [40]:
test_celeb_dataset = datasets.ImageFolder(root='../CelebDF_v2/extracted_frames', transform=transform)
celeb_loader = DataLoader(test_celeb_dataset, batch_size=256, shuffle=False, num_workers=2)

# 測試模型
model.load_state_dict(torch.load('best_efficientformerV2_s0.pth'))
model.eval()
test_running_corrects = 0

with torch.no_grad():
    for inputs, labels in celeb_loader:
        inputs = inputs.to(device)
        labels = labels.to(device)

        outputs = model(inputs)
        _, preds = torch.max(outputs, 1)
        test_running_corrects += torch.sum(preds == labels.data)

test_acc = test_running_corrects.double() / len(test_dataset)
print(f'Test Acc of Celeb DF V2: {test_acc:.4f}')

Test Acc of Celeb DF V2: 0.7100


In [37]:
# 使用 TorchScript 將模型保存為 .pt 文件
import torch
import timm

# 加載預訓練的 MobileNetV3 大模型
model = timm.create_model('efficientformerv2_s0', pretrained=False)
model.head = nn.Linear(model.head.in_features, 2)
model.head_dist = nn.Linear(model.head_dist.in_features, 2)
model.load_state_dict(torch.load('best_efficientformerV2_s0.pth'))
model.eval()

example_input = torch.rand(1, 3, 224, 224)
traced_script_module = torch.jit.trace(model, example_input)
torch.jit.save(traced_script_module, 'efficientformerV2_jit.pt')